## Notebook overview <a class="tocSkip">

## Install additional R packages

Many notebooks reference additional packages and libraries, which you will install in this section.  

In [1]:
# The bigrquery package makes it easy to work with data stored in Google BigQuery by allowing you to query 
# BigQuery tables and retrieve metadata about your projects, datasets, tables, and jobs
library(bigrquery)

# ggplot2 is a library of integrated plotting functions
library(ggplot2)

Next we need to allow the notebook kernel, which is on a virtual machine, to speak to the workspace, which is mounted to a google bucket. The program that does this, called FISS (Firecloud Service Selector), is a Python module that allows API (Application Programming Interface) calls from the notebook to the workspace.     

In [2]:
# The reticulate package translates Python code into R code, so we can use Python commands in 
# this R notebook (remember FISS is a Python module)
library(reticulate)

In [4]:
# Import Firecloud Service Selector (FISS) modules using reticulate
os <- import("os")
firecloud <- import("firecloud")
fiss <- import("firecloud.fiss")

## Set project globals

These next few code cells set global variables, like the workspace name, automatically. This way, the notebook can be used in multiple workspaces with different project names and different billing projects without any manual adjustments.

In [6]:
# Set the project variables. Using variables ensures this notebook works without having to manually
# retype the project or workspace names (when copied to another workspace, for example)
PROJECT <- Sys.getenv('GOOGLE_PROJECT')
WORKSPACE <- os$path$basename(os$path$dirname(os$getcwd()))

In [13]:
# Set the project ID of the clould project to bill for queries to BigQuery
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

In [7]:
# To use access BigQuery, you must first autheniticate, or verify your identity.
# This code cell does that
bq_auth(path=WORKSPACE)

# Get the cohort query

In this section, we are going to pull information from the "cohort" table you created at the beginning of this tutorial into the notebook environment. We'll use the workspace variables and FISS to accomplish this. 

In [8]:
# The FISS module that communicates between the notebook environment and Terra is in Python
# So the first step is to create Python versions of the workspace and project variables
workspace = r_to_py(WORKSPACE)
project = r_to_py(PROJECT)

In [9]:
# Bring information from the "cohort" table into a notebook variable "cohort_entity".
cohort_entity<- fiss$fapi$get_entities(project,workspace,"cohort")$json()

In [10]:
# From the first row in the `cohort` table, find the column with the header "query" and save this information 
# as a variable "cohort_query"     
cohort_query <- cohort_entity[[1]][['attributes']][['query']]

In [11]:
# Sanity check - This cell just prints out the cohort_entity variable - information the notebook will use
# The values should look familiar!
cohort_entity

[[1]]
[[1]]$attributes
[[1]]$attributes$query
[1] "SELECT DISTINCT t1.participant_id FROM ((SELECT participant_id FROM `verily-public-data.human_genome_variants.1000_genomes_participant_info` WHERE (Super_Population_Description = \"European\")) INTERSECT DISTINCT (SELECT participant_id FROM `verily-public-data.human_genome_variants.1000_genomes_participant_info` WHERE (Population_Description = \"British in England and Scotland\"))) t1"

[[1]]$attributes$data_explorer_url
[1] "https://app.terra.bio/#library/datasets/1000 Genomes/data-explorer?filter=verily-public-data.human_genome_variants.1000_genomes_participant_info.Super_Population_Description%253DEuropean%257Cverily-public-data.human_genome_variants.1000_genomes_participant_info.Population_Description%253DBritish+in+England+and+Scotland&extraFacets="

[[1]]$attributes$dataset_name
[1] "1000 Genomes"


[[1]]$entityType
[1] "cohort"

[[1]]$name
[1] "1000G-British"

### <font color="#FF6600">(expand for tip) </font> <font color="#445555"> How to select from multiple workspace cohorts <a class="tocSkip">

<font color="#445555">The command you just ran grabs the first cohort in your workspace data table. If there were multiple queries saved into your workspace table, the one you choose can be changed by updating the number in the double brackets in the code cell above.   

For example, using [[2]] would point to the cohort in the second row of the table.  

# Call BigQuery

In [14]:
# Execute the query and return all results into an in-memory table, "t", in R
t <- bigrquery::bq_project_query(
    BILLING_PROJECT_ID,
    cohort_query
)
print(t)
tt <- bigrquery::bq_table_download(t)

<bq_table> broad-cpa-pipeline._a842069f02d1a3fa5cb5920da81a31ddb8587d9b.anon08101cd6ed030da1ffd5c14c29edaafb913b475d


In [15]:
# Examine the output. This is a table of the participant IDs of your cohort. 
print(tt)

# A tibble: 107 × 1
   participant_id
   <chr>         
 1 HG04302       
 2 HG00098       
 3 HG00152       
 4 HG00156       
 5 HG00145       
 6 HG00157       
 7 HG00115       
 8 HG01789       
 9 HG01791       
10 HG00105       
# … with 97 more rows


# Add data by "joining" with another table
The query you just ran returns **just the participant IDs** of your cohort in a table. To do an analysis, you will want additional information from the BigQuery dataset. This section is an example of how to access the additional data you need. 

In [ ]:
# Query the public-access data to grab pairs of participants IDs and genders and put in a table (named "table") 
query <- '
SELECT
    DISTINCT participant_id,
    Gender
FROM
    `verily-public-data.human_genome_variants.1000_genomes_participant_info`
'
table_data <- bigrquery::bq_project_query(
    BILLING_PROJECT_ID,
    query
)
table <- bigrquery::bq_table_download(table_data)

In [ ]:
# Print the dimensions of `table`, which includes all participants, not just those in your cohort 
dim(table)

In [ ]:
# Create a **new table**, `merged_table`, of the participant ID and gender pairs for the participants in the cohort
merged_table <- merge(x = tt, y = table, by="participant_id", all.x = TRUE)

In [ ]:
# Sanity check - print out the dimensions of the merged table 
# It should have two columns (for ID and gender) and the number of rows = size of your cohort
dim(merged_table)

# Plot data

In [ ]:
# This plot is to confirm you hve the data you expect in your notebook 
grouped <- table(merged_table$Gender)
print(grouped)

g <- ggplot(merged_table, aes(Gender))
g + geom_bar()

## <font color="#445555">How to make your own analysis</font><a class="tocSkip">

<font color="#445555">This notebook is a demonstration - not really an analysis at all. If you were doing an actual analysis, this is where you would include your R or Python analysis code.</font>

# Provenance

Provenance is a record of exactly the environment you used when running the notebook. It's useful for collaborating, and helpful when you return to a notebook months after your initial analysis. It's also Best Practices for reproducible research.

In [ ]:
# Outputs all session information
devtools::session_info()

Copyright 2019 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.